In [1]:
import numpy as np
import pandas as pd 
import evidently
from evidently.report import Report
from evidently.metric_preset import DataDriftPreset
from evidently import ColumnMapping
import time
from datetime import timedelta

#On importe data_final et data_test, puis on retire les colonnes ID et target
data_final = pd.read_parquet('../Artefacts/data_final.parquet')
data_final = data_final.drop(columns=['TARGET', 'SK_ID_CURR'])
data_test = pd.read_parquet('../Artefacts/data_test.parquet')
data_test = data_test.drop(columns=['TARGET', 'SK_ID_CURR'])

#Nos colonnes catégorielles ont été one-hot encodées, ce qui veut dire qu'on peut facilement les retrouver en isolant les colonnes qui ne peuvent prendre que les valeurs 0, 1 ou NaN
cols_categ = [col for col in data_final.columns if data_final[col].dropna().isin([0, 1]).all()]

#Le reste des colonnes est forcément numérique
cols_num = [col for col in data_final.columns if col not in cols_categ]

#Début du timer
start_time = time.time()

#Normalement c'est bon, mais on vérifie que les 2 dfs ont bien les mêmes colonnes
assert set(data_final.columns) == set(data_test.columns)
print('Les colonnes correspondent !') #Si assertion réussie

#On créé un column mapping
column_mapping = ColumnMapping()
column_mapping.features_categ = cols_categ
column_mapping.features_num = cols_num

#On créé le rapport de data drift
print('Création du rapport')
data_drift_report = Report(metrics=[DataDriftPreset(num_stattest='ks', cat_stattest='psi', num_stattest_threshold=0.2, cat_stattest_threshold=0.2)])

#On exécute l'analyse
print("Exécution de l'analyse")
data_drift_report.run(reference_data=data_final, current_data=data_test, column_mapping=column_mapping)

#Fin du timer
end_time = time.time()
temps_total = end_time - start_time
temps_total = timedelta(seconds=int(temps_total))
print('Temps total :', temps_total)

#On enregistre le rapport en tant que fichier html
print('Enregistrement du rapport')
data_drift_report.save_html('../../Veynachter_Quentin_4_Tableau_HTML_Data_Drift_Evidently_112023.html')

Les colonnes correspondent !
Création du rapport
Exécution de l'analyse
Temps total : 0:22:49
Enregistrement du rapport
